In [269]:
import pandas as pd
import numpy as np
import matplotlib
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats.distributions as dist
matplotlib.use('Agg') # workaround, there may be a better way

In [241]:
da = pd.read_csv("C:/Users/eli/Desktop/nhanes_2015_2016.csv")
da.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
0,83732,1.0,NaN,1.0,1,1,62,3,1.0,5.0,...,124.0,64.0,94.8,184.5,27.8,43.3,43.6,35.9,101.1,2.0
1,83733,1.0,NaN,6.0,1,1,53,3,2.0,3.0,...,140.0,88.0,90.4,171.4,30.8,38.0,40.0,33.2,107.9,NaN
2,83734,1.0,NaN,NaN,1,1,78,3,1.0,3.0,...,132.0,44.0,83.4,170.1,28.8,35.6,37.0,31.0,116.5,2.0
3,83735,2.0,1.0,1.0,2,2,56,3,1.0,5.0,...,134.0,68.0,109.8,160.9,42.4,38.5,37.7,38.3,110.1,2.0
4,83736,2.0,1.0,1.0,2,2,42,4,1.0,4.0,...,114.0,54.0,55.2,164.9,20.3,37.4,36.0,27.2,80.4,2.0


In [242]:
da["SMQ020"] = da.SMQ020.map({1: "Yes", 2: "No"})
da["SMQ020"].head()

0    Yes
1    Yes
2    Yes
3     No
4     No
Name: SMQ020, dtype: object

In [243]:
da["RIAGENDR"] = da.RIAGENDR.replace({1: "Male", 2: "Female"})
da["RIAGENDR"].head()

0      Male
1      Male
2      Male
3    Female
4    Female
Name: RIAGENDR, dtype: object

# Hypothesis Tests for One Proportion

The most basic hypothesis test may be the one-sample test for a proportion. This test is used if we have specified a particular value as the null value for the proportion, and we wish to assess if the data are compatible with the true parameter value being equal to this specified value. One-sample tests are not used very often in practice, because it is not very common that we have a specific fixed value to use for comparison. For illustration, imagine that the rate of lifetime smoking in another country was known to be 40%, and we wished to assess whether the rate of lifetime smoking in the US were different from 40%. In the following notebook cell, we carry out the (two-sided) one-sample test that the population proportion of smokers is 0.4, and obtain a p-value of 0.50. This indicates that the NHANES data are compatible with the proportion of (ever) smokers in the US being 40%.


In [244]:
condition = da["SMQ020"] == 'Yes'
da_smokers = da[condition]
n_smokers = len(da_smokers)
n_smokers

2319

In [245]:
p_null = 0.40
p_hat = n_smokers / len(da["SMQ020"])
n = len(da["SMQ020"])
print(n, p_hat)

5735 0.404359197907585


In [246]:
sm.stats.proportions_ztest(p_hat * n, n, p_null)

(0.672662805775994, 0.501161835318324)

# Another way:

In [247]:
Z_test_statistic = (p_hat - p_null) / np.sqrt((p_null * (1 - p_null)) / n)
Z_test_statistic

0.673856895322319

In [248]:
pvalue = 2 * dist.norm.cdf(-np.abs(Z_test_statistic))
print(Z_test_statistic, pvalue)

0.673856895322319 0.5004022987340593


# Hypothesis Tests for Two Proportions

Comparative tests tend to be used much more frequently than tests comparing one population to a fixed value. A two-sample test of proportions is used to assess whether the proportion of individuals with some trait differs between two sub-populations. For example, we can compare the smoking rates between females and males. Since smoking rates vary strongly with age, we do this in the subpopulation of people between 20 and 25 years of age. We find that the smoking rate for men is around 10 percentage points greater than the smoking rate for females, and this difference is statistically significant (the p-value is around 0.009).


In [249]:
condition1 = da['RIDAGEYR'] >= 20 
condition2 = da['RIDAGEYR'] <= 25
df_20_25 = da[condition1 & condition2]
df_20_25.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
6,83741,1.0,NaN,8.0,Yes,Male,22,4,1.0,4.0,...,112.0,74.0,76.6,165.4,28.0,38.8,38.0,34.0,86.6,NaN
17,83761,1.0,NaN,1.0,No,Female,24,5,2.0,5.0,...,108.0,60.0,61.8,156.4,25.3,37.0,35.5,29.6,79.5,NaN
26,83784,1.0,NaN,4.0,Yes,Male,22,2,1.0,4.0,...,124.0,84.0,73.7,170.7,25.3,41.2,36.0,29.7,86.2,2.0
38,83809,2.0,2.0,NaN,No,Female,20,4,1.0,3.0,...,100.0,62.0,72.9,166.8,26.2,39.2,36.6,31.6,92.6,NaN
40,83813,1.0,NaN,2.0,Yes,Male,24,3,1.0,4.0,...,124.0,68.0,89.2,182.2,26.9,44.2,40.2,33.9,89.2,2.0


In [313]:
male = df_20_25.loc[df_20_25['RIAGENDR'] == 'Male']
male.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
6,83741,1.0,NaN,8.0,1.0,Male,22,4,1.0,4.0,...,112.0,74.0,76.6,165.4,28.0,38.8,38.0,34.0,86.6,NaN
26,83784,1.0,NaN,4.0,1.0,Male,22,2,1.0,4.0,...,124.0,84.0,73.7,170.7,25.3,41.2,36.0,29.7,86.2,2.0
40,83813,1.0,NaN,2.0,1.0,Male,24,3,1.0,4.0,...,124.0,68.0,89.2,182.2,26.9,44.2,40.2,33.9,89.2,2.0
48,83824,NaN,NaN,NaN,0.0,Male,23,5,2.0,5.0,...,124.0,84.0,72.4,180.3,22.3,41.4,34.6,28.7,80.7,2.0
96,83907,NaN,NaN,NaN,0.0,Male,22,5,2.0,5.0,...,132.0,80.0,71.5,172.4,24.1,39.8,34.0,28.1,86.0,2.0


In [312]:
female = df_20_25.loc[df_20_25['RIAGENDR'] == 'Female']
female.head()

,SEQN,ALQ101,ALQ110,ALQ130,SMQ020,RIAGENDR,RIDAGEYR,RIDRETH1,DMDCITZN,DMDEDUC2,...,BPXSY2,BPXDI2,BMXWT,BMXHT,BMXBMI,BMXLEG,BMXARML,BMXARMC,BMXWAIST,HIQ210
17,83761,1.0,NaN,1.0,0.0,Female,24,5,2.0,5.0,...,108.0,60.0,61.8,156.4,25.3,37.0,35.5,29.6,79.5,NaN
38,83809,2.0,2.0,NaN,0.0,Female,20,4,1.0,3.0,...,100.0,62.0,72.9,166.8,26.2,39.2,36.6,31.6,92.6,NaN
46,83822,2.0,2.0,NaN,0.0,Female,20,4,1.0,4.0,...,116.0,60.0,60.4,164.8,22.2,38.6,37.5,25.5,78.4,NaN
69,83865,1.0,NaN,4.0,1.0,Female,21,3,1.0,2.0,...,90.0,60.0,39.7,156.3,16.3,36.1,34.1,20.2,65.5,2.0
102,83914,1.0,NaN,6.0,1.0,Female,23,2,1.0,4.0,...,112.0,60.0,99.3,171.5,33.8,42.3,34.8,35.3,115.4,1.0


In [253]:
n_male_total = len(male)
n1_smoke = len(male[male['SMQ020'] == 'Yes'])
p_hat_male_smoker = n1_smoke / n_male_total
p_hat_male_smoker

0.3412698412698413

In [254]:
n_female_total = len(female)
n2_smoke = len(female[female['SMQ020'] == 'Yes'])
p_hat_female_smoker = n2_smoke / n_female_total
p_hat_female_smoker

0.23809523809523808

# Another way to calculate proportions

In [343]:
df_20_25["SMQ020"] = df_20_25.SMQ020.replace({"Yes": 1, "No": 0})
dz = df_20_25.groupby("RIAGENDR").agg({"SMQ020": [np.mean, np.size]})
dz.columns = ["Proportion of smokers", "Total n"]
dz

C:\Users\eli\AppData\Local\Temp/ipykernel_1288/1090486549.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_20_25["SMQ020"] = df_20_25.SMQ020.replace({"Yes": 1, "No": 0})


,Proportion of smokers,Total n
RIAGENDR,,
Female,0.238971,273
Male,0.341270,252


In [257]:
p_hat_total = (n1 + n2) / (n_male_total + n_female_total)
best_est = p_hat_male_smoker - p_hat_female_smoker

Z_test_stats = best_est / np.sqrt(p_hat_total * (1 - p_hat_total) * (1/n_male_total + 1/n_female_total))
Z_test_stats

2.6092144683138088

In [258]:
# Calculate the P-value
pvalue = 2*dist.norm.cdf(-np.abs(Z_test_stats)) # Multiplied by two indicates a two tailed testing.
print("Computed P-value is", pvalue)

Computed P-value is 0.00907503457584406


# Hypothesis Tests Comparing Means

Tests of means are similar in many ways to tests of proportions. Just as with proportions, for comparing means there are one and two-sample tests, z-tests and t-tests, and one-sided and two-sided tests. As with tests of proportions, one-sample tests of means are not very common, but we illustrate a one sample test in the cell below. We compare systolic blood pressure to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).
We compare systolic blood pressure to the fixed value 120 (which is the lower threshold for "pre-hypertension"), and find that the mean is significantly different from 120 (the point estimate of the mean is 126).

In [298]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDR"]].dropna()
dx

,BPXSY1,RIDAGEYR,RIAGENDR
0,128.0,62,Male
1,146.0,53,Male
2,138.0,78,Male
3,132.0,56,Female
4,100.0,42,Female
...,...,...,...
5730,112.0,76,Female
5731,118.0,26,Male
5732,154.0,80,Female
5733,104.0,35,Male


In [300]:
dx = dx.loc[(dx.RIDAGEYR >= 40) & (dx.RIDAGEYR <= 50) & (dx.RIAGENDR == "Male"), :]
dx

,BPXSY1,RIDAGEYR,RIAGENDR
10,144.0,46,Male
11,116.0,45,Male
20,110.0,49,Male
42,128.0,42,Male
51,118.0,50,Male
...,...,...,...
5680,134.0,50,Male
5690,138.0,48,Male
5693,96.0,41,Male
5713,116.0,43,Male


In [301]:
print(dx.BPXSY1.mean())

125.86698337292161


In [302]:
sm.stats.ztest(dx.BPXSY1, value=120)

(7.469764137102597, 8.033869113167905e-14)

For age 50-60 the sistolic blood presure is different from 120.



In the cell below, we carry out a formal test of the null hypothesis that the mean blood pressure for women between the ages of 50 and 60 is equal to the mean blood pressure of men between the ages of 50 and 60. The results indicate that while the mean systolic blood pressure for men is slightly greater than that for women (129 mm/Hg versus 128 mm/Hg), this difference is not statistically significant.

There are a number of different variants on the two-sample t-test. Two often-encountered variants are the t-test carried out using the t-distribution, and the t-test carried out using the normal approximation to the reference distribution of the test statistic, often called a z-test. Below we display results from both these testing approaches. When the sample size is large, the difference between the t-test and z-test is very small.


In [304]:
dx = da[["BPXSY1", "RIDAGEYR", "RIAGENDR"]].dropna()
dx = dx.loc[(dx.RIDAGEYR >= 50) & (dx.RIDAGEYR <= 60), :]
dx.head()

,BPXSY1,RIDAGEYR,RIAGENDR
1,146.0,53,Male
3,132.0,56,Female
9,178.0,56,Male
15,134.0,57,Female
19,136.0,54,Female


In [306]:
bpx_female = dx.loc[dx.RIAGENDR=="Female", "BPXSY1"]
bpx_male = dx.loc[dx.RIAGENDR=="Male", "BPXSY1"]
print(bpx_female.mean(), bpx_male.mean())

127.92561983471074 129.23829787234044


In [362]:
print(sm.stats.ztest(bpx_female, bpx_male))

(-1.105435895556249, 0.2689707570859362)


In [363]:
print(sm.stats.ttest_ind(bpx_female, bpx_male))

(-1.105435895556249, 0.26925004137768577, 952.0)


Another important aspect of two-sample mean testing is "heteroscedasticity", meaning that the variances within the two groups being compared may be different. While the goal of the test is to compare the means, the variances play an important role in calibrating the statistics (deciding how big the mean difference needs to be to be declared statisitically significant). In the NHANES data, we see that there are moderate differences between the amount of variation in BMI for females and for males, looking within 10-year age bands. In every age band, females having greater variation than males. 

In [365]:
dx = da[["BMXBMI", "RIDAGEYR", "RIAGENDR"]].dropna()
da["agegrp"] = pd.cut(da.RIDAGEYR, [18, 30, 40, 50, 60, 70, 80])
da.groupby(["agegrp", "RIAGENDR"])["BMXBMI"].agg(np.std).unstack()

RIAGENDR,Female,Male
agegrp,,
"(18, 30]",7.745893,6.649440
"(30, 40]",8.315608,6.622412
"(40, 50]",8.076195,6.407076
"(50, 60]",7.575848,5.914373
"(60, 70]",7.604514,5.933307
"(70, 80]",6.284968,4.974855


The standard error of the mean difference (e.g. mean female blood pressure minus mean mal blood pressure) can be estimated in at least two different ways. In the statsmodels library, these approaches are referred to as the "pooled" and the "unequal" approach to estimating the variance. If the variances are equal (i.e. there is no heteroscedasticity), then there should be little difference between the two approaches. Even in the presence of moderate heteroscedasticity, as we have here, we can see that the results for the two differences are quite similar. Below we have a loop that considers each 10-year age band and assesses the evidence for a difference in mean BMI for women and for men. The results printed in each row of output are the test-statistic and p-value.


In [367]:
for k, v in da.groupby("agegrp"):
    bmi_female = v.loc[v.RIAGENDR=="Female", "BMXBMI"].dropna()
    bmi_female = sm.stats.DescrStatsW(bmi_female)
    bmi_male = v.loc[v.RIAGENDR=="Male", "BMXBMI"].dropna()
    bmi_male = sm.stats.DescrStatsW(bmi_male)
    print(k)
    print("pooled: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='pooled'))
    print("unequal: ", sm.stats.CompareMeans(bmi_female, bmi_male).ztest_ind(usevar='unequal'))
    print()

(18, 30]
pooled:  (1.702693293364314, 0.08862548061450115)
unequal:  (1.7174610823927015, 0.08589495934713479)

(30, 40]
pooled:  (1.4378280405644988, 0.15048285114647975)
unequal:  (1.4437869620833568, 0.1487989105789227)

(40, 50]
pooled:  (2.8933761158070186, 0.003811246059501354)
unequal:  (2.9678691663536725, 0.0029987194174035366)

(50, 60]
pooled:  (3.3621087799813747, 0.0007734964571391533)
unequal:  (3.375494390173931, 0.0007368319423226358)

(60, 70]
pooled:  (3.6172401442432602, 0.0002977610210319532)
unequal:  (3.628483094544545, 0.0002850914147149385)

(70, 80]
pooled:  (2.926729252512241, 0.003425469414486057)
unequal:  (2.9377798867692064, 0.0033057163315194853)

